In [1]:
from preprocessing import get_entities

get_entities()

2021-09-21 13:15:47 WARNING: Can not find mwt: default from official model list. Ignoring it.


Retrieved Stanza document (688 words, 688 tokens).
Identified 10654 eventive nouns.
Extracting entities...


[['Detectives', 0, 1, 'seized', 3, 'nsubj', '', -1],
 ['sum', 0, 6, 'seized', 3, 'obj', 'large', 5],
 ['drugs', 0, 11, 'seized', 3, 'obl', '', -1],
 ['designer', 0, 14, 'watch', 15, 'compound', '', -1],
 ['associate', 0, 20, 'targeting', 17, 'obj', 'female', 19],
 ['lord', 0, 23, 'associate', 20, 'nmod', '', -1],
 ['gardai', 1, 2, 'raided', 3, 'nsubj', 'Armed', 1],
 ['property', 1, 7, 'raided', 3, 'obj', '', -1],
 ['evening', 1, 10, 'raided', 3, 'obl', '', -1],
 ['part', 1, 12, 'raided', 3, 'obl', 'investigation', 16],
 ['investigation', 1, 16, 'part', 12, 'nmod', 'ongoing', 15],
 ['mob', 1, 20, 'investigation', 16, 'nmod', 'Rattigan', 18],
 ['killer', 2, 3, 'suspected', 16, 'nsubj:pass', 'convicted', 2],
 ['term', 2, 10, 'serving', 7, 'obj', 'Prison', 13],
 ['drugs', 2, 22, 'operation', 23, 'compound', '', -1],
 ['operation', 2, 23, 'run', 20, 'obj', 'bars', 25],
 ['bars', 2, 25, 'operation', 23, 'nmod', '', -1],
 ['mob', 2, 29, 'believed', 31, 'nsubj:pass', 'his', 28],
 ['network', 2